![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Train Finance Assertion

In [0]:
from johnsnowlabs import * 

#Data Prep

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/assertion_fin.csv

In [0]:
import pandas as pd

training_df = pd.read_csv('assertion_fin.csv')

In [0]:
training_data = spark.createDataFrame(training_df)
training_data.show()

+-------+--------------------+---------+-------+--------------------+------+---------------+
task_id| sentence|tkn_start|tkn_end| chunk|entity|assertion_label|
+-------+--------------------+---------+-------+--------------------+------+---------------+
 1|The Swedish East ...| 1| 4|Swedish East Indi...| ORG| PAST|
 1|The Swedish East ...| 6| 8|Svenska Ostindisk...| ALIAS| PAST|
 1|The Swedish East ...| 10| 10| SOIC| ALIAS| PAST|
 1|The Swedish East ...| 14| 14| Gothenburg| LOC| PAST|
 1|The Swedish East ...| 15| 15| Sweden| LOC| PAST|
 1|The Swedish East ...| 17| 17| 1731| DATE| PAST|
 1|The Swedish East ...| 25| 25| China| LOC| PAST|
 1|The Swedish East ...| 28| 29| Far East| LOC| PAST|
 1|The venture was i...| 9| 12|Dutch East India ...| ORG| PAST|
 1|The venture was i...| 15| 18|British East Indi...| ORG| PAST|
 1|This made Gothenb...| 2| 2| Gothenburg| LOC| PAST|
 1|Trade with China ...| 2| 2| China| LOC| PAST|
 1|Trade with China ...| 11| 11| Sweden| LOC| PAST|
 1|The Chinese cultu...| 34| 34| Sweden| LOC| PAST|
 1|The company folde...| 4| 4| 1813| DATE| PAST|
 1|nevertheless, it ...| 11| 11| Gothenburg| LOC| PAST|
 1|Background Sweden...| 16| 17| East India| LOC| PAST|
 1|The royal privile...| 5| 8|Swedish East Indi...| ORG| PAST|
 1|The royal privile...| 9| 9| SOIC| ALIAS| PAST|
 1|The royal privile...| 27| 28| East India| LOC| PAST|
+-------+--------------------+---------+-------+--------------------+------+---------------+
only showing top 20 rows

In [0]:
training_data.printSchema()

root
-- task_id: long (nullable = true)
-- sentence: string (nullable = true)
-- tkn_start: long (nullable = true)
-- tkn_end: long (nullable = true)
-- chunk: string (nullable = true)
-- entity: string (nullable = true)
-- assertion_label: string (nullable = true)

In [0]:
%time training_data.count()

CPU times: user 842 µs, sys: 446 µs, total: 1.29 ms
Wall time: 1.05 s
Out[6]: 8050

In [0]:
(train_data, test_data) = training_data.randomSplit([0.9, 0.1], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 8050
Test Dataset Count: 797

In [0]:
train_data.show()

+-------+--------------------+---------+-------+--------------------+------+---------------+
task_id| sentence|tkn_start|tkn_end| chunk|entity|assertion_label|
+-------+--------------------+---------+-------+--------------------+------+---------------+
 1|"Stockholms-varve...| 6| 6| Stockholm| LOC| PAST|
 1|"The funny busine...| 5| 8|Swedish East Indi...| ORG| PAST|
 1| (1998).| 0| 0| 1998| DATE| PAST|
 1|2.5 tonnes) and t...| 34| 34| Sweden| LOC| PAST|
 1|37. Gothenburg: R...| 2| 7|Royal Society of ...| ORG| PAST|
 1|= Decline and fal...| 11| 11| 1806| DATE| PAST|
 1|= Early attempts ...| 9| 11| Swedish East India| ORG| PAST|
 1|= Early attempts ...| 19| 19| merchant| ROLE| PAST|
 1|= Early attempts ...| 20| 21| Willem Usselincx| PER| PAST|
 1|= Sweden after th...| 1| 1| Sweden| LOC| PAST|
 1|= The Royal chart...| 9| 12|Henrik König & Co...| ORG| PAST|
 1|= The Royal chart...| 39| 42| Cape of Good Hope| LOC| PAST|
 1|= The Royal chart...| 46| 46| Japan| LOC| PAST|
 1|= The Royal chart...| 79| 79| Gothenburg| LOC| PAST|
 1|= The first octro...| 8| 8| directors| ROLE| PAST|
 1|= The first octro...| 11| 11| SOIC| ORG| PAST|
 1|= The first octro...| 13| 14| Henrik König| PER| PAST|
 1|= The first octro...| 15| 16| Colin Campbell| PER| PAST|
 1|= The first octro...| 23| 23| Stockholm| LOC| PAST|
 1|= The fourth octr...| 6| 6| SOIC| ORG| PAST|
+-------+--------------------+---------+-------+--------------------+------+---------------+
only showing top 20 rows

#Using Bert Embeddings

In [0]:
bert_embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base", "en") \
  .setInputCols("document", "token") \
  .setOutputCol("embeddings")\
  .setMaxSentenceLength(512)

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
document = nlp.DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")

chunk = nlp.Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("doc_chunk")\
    .setChunkCol("chunk")\
    .setStartCol("tkn_start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(False)

token = nlp.Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')


We save the test data in parquet format to use in `AssertionDLApproach()`.

In [0]:
assertion_pipeline = nlp.Pipeline(
    stages = [
    document,
    chunk,
    token,
    bert_embeddings])

assertion_test_data = assertion_pipeline.fit(test_data).transform(test_data)

In [0]:
assertion_test_data.columns

Out[12]: ['task_id',
 'sentence',
 'tkn_start',
 'tkn_end',
 'chunk',
 'entity',
 'assertion_label',
 'document',
 'doc_chunk',
 'token',
 'embeddings']

In [0]:
assertion_test_data.write.mode('overwrite').parquet('test_data.parquet')

In [0]:
assertion_train_data = assertion_pipeline.fit(training_data).transform(training_data)
assertion_train_data.write.mode('overwrite').parquet('train_data.parquet')

In [0]:
assertion_train_data.columns

Out[15]: ['task_id',
 'sentence',
 'tkn_start',
 'tkn_end',
 'chunk',
 'entity',
 'assertion_label',
 'document',
 'doc_chunk',
 'token',
 'embeddings']

## Graph setup

We will use TFGraphBuilder annotator which can be used to create graphs in the model training pipeline. 

TFGraphBuilder inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [0]:
graph_folder= "tf_graphs"

In [0]:
assertion_graph_builder =  finance.TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("assertion_label")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(1200)\
    .setHiddenUnitsNumber(25)

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

In [0]:
scope_window = [50, 50]

assertionStatus = finance.AssertionDLApproach()\
    .setLabelCol("assertion_label")\
    .setInputCols("document", "doc_chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setEpochs(2)\
    .setStartCol("tkn_start")\
    .setEndCol("tkn_end")\
    .setMaxSentLen(1200)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('training_logs/')\
    .setGraphFolder(graph_folder)\
    .setGraphFile(f"{graph_folder}/assertion_graph.pb")\
    .setTestDataset(path="dbfs:/test_data.parquet", read_as='SPARK', options={'format': 'parquet'})\
    .setScopeWindow(scope_window)
    #.setValidationSplit(0.2)\    
    #.setDropout(0.1)\    

In [0]:
finance_assertion_pipeline = nlp.Pipeline(
    stages = [
    #document,
    #chunk,
    #token,
    #embeddings,
    assertion_graph_builder,
    assertionStatus])

In [0]:
training_data.printSchema()

root
-- task_id: long (nullable = true)
-- sentence: string (nullable = true)
-- tkn_start: long (nullable = true)
-- tkn_end: long (nullable = true)
-- chunk: string (nullable = true)
-- entity: string (nullable = true)
-- assertion_label: string (nullable = true)

In [0]:
assertion_train_data = spark.read.parquet('dbfs:/train_data.parquet')

In [0]:
%%time
assertion_model = finance_assertion_pipeline.fit(assertion_train_data)

TF Graph Builder configuration:
Model name: assertion_dl
Graph folder: tf_graphs
Graph file name: assertion_graph.pb
Build params: {'n_classes': 4, 'feat_size': 768, 'max_seq_len': 1200, 'n_hidden': 25}
04:37:06, WARNING From /databricks/python/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:111: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Device mapping: no known devices.
04:37:06, WARNING From /databricks/python/lib/python3.8/site-packages/sparknlp_jsl/_tf_graph_builders/tf2contrib/rnn.py:229: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
04:37:06, WARNING From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/rnn.py:441: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
04:37:06, WARNING From /databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:766: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Device mapping: no known devices.
assertion_dl graph exported to tf_graphs/assertion_graph.pb
CPU times: user 11.6 s, sys: 2.26 s, total: 13.9 s
Wall time: 22min 28s

In [0]:
assertion_test_data = spark.read.parquet('dbfs:/test_data.parquet')

In [0]:
preds = assertion_model.transform(assertion_test_data).select('assertion_label','assertion.result')

preds.show()

+---------------+---------+
assertion_label| result|
+---------------+---------+
 PAST| [PAST]|
 PAST| [PAST]|
 PAST|[PRESENT]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PRESENT|[PRESENT]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
+---------------+---------+
only showing top 20 rows

In [0]:
preds_df = preds.toPandas()

In [0]:
preds_df["result"] = preds_df["result"].apply(lambda x: x[0] if len(x) else pd.NA)
preds_df.dropna(inplace=True)

preds_df

Out[26]:

,assertion_label,result
0,PAST,PAST
1,PAST,PAST
2,PAST,PRESENT
3,PAST,PAST
4,PAST,PAST
...,...,...
792,POSSIBLE,POSSIBLE
793,POSSIBLE,POSSIBLE
794,POSSIBLE,POSSIBLE
795,POSSIBLE,POSSIBLE


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report( preds_df['assertion_label'], preds_df['result']))

precision recall f1-score support

 FUTURE 0.86 0.97 0.91 123
 PAST 0.92 0.93 0.92 280
 POSSIBLE 0.99 0.89 0.94 177
 PRESENT 0.91 0.91 0.91 217

 accuracy 0.92 797
 macro avg 0.92 0.92 0.92 797
weighted avg 0.92 0.92 0.92 797

###Saving the trained model

In [0]:
assertion_model.stages

Out[28]: [TFGraphBuilderModel_135f1df9637c, FINANCE-ASSERTION_DL_a1df33366aaf]

In [0]:
# Save a Spark NLP model
assertion_model.stages[-1].write().overwrite().save('/dbfs/Assertion')
